In [1]:
import zipfile

with zipfile.ZipFile("drive/My Drive/ANN/SR/Spectrogram.zip","r") as z:
    z.extractall(".")

In [ ]:
#!pip install keras
#!pip install tensorflow
#!pip install opencv-python
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D


from keras import callbacks
from keras import optimizers

import os

import keras.models

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

import tensorflow as tf
import cv2
import sys

from keras import backend as K

from keras.utils import np_utils
from IPython.display import clear_output

def clear_output():  
    clear_output()

def cnn_TrainTest(no_of_epochs, no_of_gpus, train_b_size, valid_b_size, data_type,  experiment_folder, input_data_dir, data_sub_type='',
                  setSize = 1, width=128, height=128, chanels = 3, test_b_size=32):

    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
    tf.compat.v1.keras.backend.set_session(sess)

    
    epochs = no_of_epochs #50-100-150
    gpu_no = no_of_gpus #1-2
    train_batch_size = train_b_size #32
    valid_batch_size = valid_b_size #32
    test_batch_size = test_b_size #32

    setDimensions = setSize #1
    img_height = width #128
    img_width = height #128
    channels = chanels #3

    class_names = []

    datatype = data_type #'Cepstr'
    exp_folder = experiment_folder #'no_of_classes_111'

    if data_sub_type =='':
        #train_data_dir = input_data_dir + datatype + '/' + exp_folder + '/train/'
        train_data_dir = input_data_dir + '/train/'
        validation_data_dir = input_data_dir + '/validate/'
        test_data_dir = input_data_dir + '/test/'
        #train_data_dir = input_data_dir
        #validation_data_dir = input_data_dir + datatype + '/' + exp_folder + '/validate/'
        #test_data_dir = input_data_dir + datatype + '/' + exp_folder + '/test/'
        path = input_data_dir + 'Results'
    else:
        train_data_dir = input_data_dir + datatype + '/' + data_sub_type +'/'+ exp_folder + '/train/'
        validation_data_dir = input_data_dir + datatype + '/' + data_sub_type +'/'+ exp_folder + '/validate/'
        test_data_dir = input_data_dir + datatype + '/' + data_sub_type +'/'+ exp_folder + '/test/'
        path = input_data_dir +'Results/' + data_sub_type


    run_name = '_' + datatype + '_' + exp_folder + '_' + str(train_batch_size) + '_' + str(valid_batch_size)

    graphPath = 'drive/My Drive/ANN/SR/' + 'Graph/' + run_name +'/'
    csvPath = 'drive/My Drive/ANN/SR/'
    checkpointerPath = 'drive/My Drive/ANN/SR/' + 'Model/'
    predictionsPath = 'drive/My Drive/ANN/SR/'

    if not os.path.exists(graphPath):
        os.makedirs(graphPath)
    if not os.path.exists(csvPath):
        os.makedirs(csvPath)
    if not os.path.exists(checkpointerPath):
        os.makedirs(checkpointerPath)
    if not os.path.exists(predictionsPath):
        os.makedirs(predictionsPath)

    
    
    csvPath = 'drive/My Drive/ANN/SR/' + run_name + '_loss.csv'
    checkpointerPath = 'drive/My Drive/ANN/SR/' + run_name + '.h5'
    summaryPath = 'drive/My Drive/ANN/SR/' + run_name + '_summary.csv'
    predictionsPath = 'drive/My Drive/ANN/SR/' + run_name + '_predictions.csv'

    class_names = [d for d in os.listdir(train_data_dir)]
    no_of_classes = len(class_names)

    train_file_no = 0
    aa = 1
    for x in class_names:
        #print(x)
        list_dir = os.path.join(train_data_dir, x)
        for name in os.listdir(list_dir):
            isfile = os.path.isfile(list_dir + '/' + name)
            if isfile:
                train_file_no = train_file_no + 1;  # count files
            if aa == 1 and isfile:  # for one time set the tensor shape
                img = cv2.imread(os.path.join(list_dir + '/', name))
                if setDimensions == 0:  # if do not set dimmensions e3xplicitly do it from the first file
                    img_height, img_width, channels = img.shape
                # set the tensor shape according to image size
                if K.image_data_format() == 'channels_first':
                    input_shape = (channels, img_width, img_height)
                else:
                    input_shape = (img_width, img_height, channels)  # tensorflow
                aa = 2

    validation_file_no = 0
    for x in class_names:
        list_dir = os.path.join(validation_data_dir, x)
        for name in os.listdir(list_dir):
            isfile = os.path.isfile(list_dir + '/' + name)
            if isfile:
                validation_file_no = validation_file_no + 1;  # count files

    test_file_no = 0
    for x in class_names:
        list_dir = os.path.join(test_data_dir, x)
        for name in os.listdir(list_dir):
            isfile = os.path.isfile(list_dir + '/' + name)
            if isfile:
                test_file_no = test_file_no + 1;  # count files

    train_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        classes=class_names,
        shuffle=True,
        class_mode='categorical',
        batch_size=train_batch_size)

    valid_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        classes=class_names,
        shuffle=True,
        batch_size=valid_batch_size,
        class_mode='categorical')

    test_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        classes=class_names,
        shuffle=False,
        batch_size=test_batch_size,
        class_mode='categorical')

    # TEST show images
    # imgs,labels = next(train_batches)
    # showImages.plots(imgs, titles=labels)



    # network topology

    # model = keras.models.load_model('drive/My Drive/ANN/SR/')
    #if (os.path.exists('drive/My Drive/ANN/SR/convmore_best_model.hdf5')):
    #    print('Loading Previous model...')
    #    model = keras.models.load_model("drive/My Drive/ANN/SR/convmore_best_model.hdf5")
    #    model.load_weights("drive/My Drive/ANN/SR/convmore_best_model.hdf5")
    #    print(model.summary())
        
    #else:
      #model = Sequential()

      #model.add(Convolution2D(32, (3, 3), input_shape=input_shape))
      #model.add(Activation('relu'))
      #model.add(BatchNormalization())
      #model.add(MaxPooling2D(pool_size=(3, 3)))

      #model.add(Convolution2D(64, (2, 2)))
      #model.add(Activation('relu'))
      #model.add(BatchNormalization())
      #model.add(MaxPooling2D(pool_size=(3, 3)))

      #model.add(Convolution2D(64, (2, 2)))
      #model.add(Activation('relu'))
      #model.add(BatchNormalization())
      #model.add(MaxPooling2D(pool_size=(2, 2)))

      #model.add(Flatten())

      #model.add(Dense(64))
      #model.add(Activation('relu'))
      #model.add(BatchNormalization())

      #model.add(Dense(no_of_classes))
      #model.add(Activation('softmax'))

    model = Sequential()
    model.add(tf.keras.layers.Reshape((128,128,3), input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"))
    model.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))

    model.add(Dense(no_of_classes))
    model.add(Activation('softmax'))

    # opt = SGD(lr=2e-3, momentum=0.9)


    batches_per_epoch = train_file_no/train_b_size
    lr_decay = (1./0.75 -1)/batches_per_epoch

    opt = optimizers.Adam(lr=1e-5,decay=lr_decay, beta_1=0.95, beta_2=0.999, epsilon=1e-08)

    print(model.summary())

    #if gpu_no > 1:
    #    model = multi_gpu.make_parallel(model, gpu_no)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # model.save('drive/My Drive/ANN/SR/')
    csv_log = callbacks.CSVLogger('drive/My Drive/ANN/SR/accuracy.csv', separator=',', append=True)

    checkpointer = callbacks.ModelCheckpoint(filepath="drive/My Drive/ANN/SR/best_model.hdf5",
                                             verbose=0, monitor='val_accuracy', 
                                             save_best_only=True, mode='max',
                                             save_weights_only=True)
    #checkpt = callbacks.ModelCheckpoint(filepath="drive/My Drive/ANN/SR/best_model.hdf5", monitor='val_accuracy', verbose=0,
    #save_weights_only=False, save_best_only=True, mode='max', period=10)
    checkpt = callbacks.ModelCheckpoint(filepath="drive/My Drive/ANN/SR/lessdropout_2dense_1conv.hdf5", monitor='val_accuracy', verbose=1,
                                        save_weights_only=False, save_best_only=True, mode='max', period=1)
    # early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta= 0, patience= 0, verbose= 0, mode= 'min')

    tbCallBack = keras.callbacks.TensorBoard(log_dir='drive/My Drive/ANN/SR/logs', histogram_freq=0, write_graph=True, write_images=True)
    
    # print(len(test_img_data))
    history = model.fit_generator(
        train_batches,
        steps_per_epoch=(train_file_no // train_batch_size + 1) * gpu_no,
        epochs=epochs,
        verbose=1,
        validation_data=valid_batches,
        validation_steps=(validation_file_no // valid_batch_size + 1) * gpu_no,
        callbacks=[checkpt, tbCallBack, csv_log]
        #callbacks=[tbCallBack, csv_log]
    )
    print(history.history.keys())
    print('\nTesting model')

    # model = load_model(path + 'Models/02ClNoNoise/' + name + '.h5')

    predictionResult = model.predict_generator(test_batches, steps=test_file_no // test_batch_size + 1, verbose=0)
    f_names = test_batches.filenames
    # test_imgs, test_labels = next(test_batches)

    tmp = test_batches.class_indices
    tmp_batch_files = test_batches.filenames
    Y_true = []
    for t in tmp_batch_files:
        for item in tmp:
            if t.split("/")[0] == item:
                Y_true.append(tmp[item])

    Y_true = np_utils.to_categorical(Y_true, no_of_classes)

    # original = sys.stdout
    file_w = open(predictionsPath, 'w')
    file_s = open(summaryPath, 'w')

    predictions = np.argmax(predictionResult, axis=1)
    file_w.write(run_name)
    file_s.write(run_name)
	
    #print(run_name, file_w)
    #print(run_name, file_s)
    #print>> file_w, run_name
    #print>> file_s, run_name


    #print("\n")
    #print("acc: " + str(history.history["acc"][epochs - 1]) + " loss: " + str(
    #    history.history["loss"][epochs - 1]) +
    #      " val_acc: " + str(history.history["val_acc"][epochs - 1]) + " val_loss: " + str(
    #    history.history["val_loss"][epochs - 1]))

    aaa = "acc: " + str(history.history["accuracy"][epochs - 1]) + " loss: " + str(
        history.history["loss"][epochs - 1]) + " val_acc: " + str(
        history.history["val_accuracy"][epochs - 1]) + " val_loss: " + str(history.history["val_loss"][epochs - 1])

    bbb = "train batch size - " + str(train_batch_size) + " validation batch size - " + str(valid_batch_size)

    file_s.write("\n")
    file_s.write("Stats")
    file_s.write(aaa)
    #print("\n", file_s)
    #print("Stats", file_s)
    #print(aaa, file_s)
    #print>> file_s, "\n"
    #print>> file_s, "Stats"
    #print>> file_s, aaa

    file_s.write("\n")
    file_s.write("Batches")
    file_s.write(aaa)
    #print("\n", file_s)
    #print("Batches", file_s)
    #print(bbb, file_s)
    #print>> file_s, "\n"
    #print>> file_s, "Batches"
    #print>> file_s, bbb

    file_s.write("\n")
    file_s.write("Clasification Report")
    #print("\n", file_s)
    #print("Clasification Report", file_s)
    #print>> file_s, "\n"
    #print>> file_s, "Classification report"
    cr = classification_report(y_true=np.argmax(Y_true, axis=1), y_pred=predictions,
                               target_names=class_names)
    file_s.write(cr)    
    print(cr, file_s)
    # print>> file_s, cr
    # print(classification_report(y_true=np.argmax(Y_true, axis=1), y_pred=predictions,
    #                            target_names=class_names))

    file_s.write("\n")
    file_s.write("Confusion Matrix")
    #print("\n", file_s)
    #print("Confusion Matrix", file_s)
    
    #print>> file_s, "\n"
    #print>> file_s, "Confusion Matrix"
    cm = confusion_matrix(np.argmax(Y_true, axis=1), predictions)
    qq = cm.tolist()
    for item in qq:
        # print(item, file_s)
        file_s.writelines(str(item))
        # print>> file_s, item
    #print>> file_s, qq
    # print(confusion_matrix(np.argmax(Y_true, axis=1), predictions))
    file_s.close()

    file_w.write("\n")
    file_w.write("Prediction Result")
    # print("\n", file_w)
    # print("Prediction Result", file_w)
    # print>> file_w, "\n"
    # print>> file_w, "Prediction Result"
    qq = predictionResult.tolist()
    for item in qq:
        # print>> file_w, item
        file_w.writelines(str(item))
        # print(item, file_w)

    file_w.write("\n")
    file_w.write("Classification Result")
    # print("\n", file_w)
    # print("Classification Result", file_w)
    # print>> file_w, "\n"
    # print>> file_w, "Classification Result"
    qq = predictionResult.tolist()
    for item in qq:
        maxVal = max(item)
        maxValIndx = item.index(max(item))
        newRow = [0] * len(item)
        newRow[maxValIndx] = 1
        # print(newRow, file_w)
        file_w.writelines(str(newRow))
        # file_w.write("Prediction Result")
        #print>> file_w, newRow

    # print("\n")
    # print("Predictions")
    # print(predictionResult)
    file_w.write("\n")
    file_w.write("True Classes")
    # print("\n", file_w)
    # print("True classes", file_w)
    # print>> file_w, "\n"
    # print>> file_w, "True classes"
    qq = Y_true.tolist()
    for item in qq:
        #print>> file_w, item
        file_w.writelines(str(item))
        # print(item, file_w)
    # print>>file_w, Y_true

    # print("\n")
    # print("True classes")
    # print(Y_true)
    file_w.write("\n")
    file_w.write("Filenames")
    # print("\n", file_w)
    # print("Filenames", file_w)
    # print>> file_w, "\n"
    # print>> file_w, "Filenames"
    for item in f_names:
        # print>> file_w, item
        file_w.write(item)
        # print(item, file_w)
    # print>>file_w, f_names

    # print("\n")
    # print("Filenames")
    # print(f_names)

    file_w.close()

    # sys.stdout = original
    sess.close()

    print('done')

cnn_TrainTest(10,1,64,32,'Spectrogram_full','no_of_classes_10','Spectrogram/' )


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0

Found 497505 images belonging to 111 classes.
Found 62085 images belonging to 111 classes.
Found 62085 images belonging to 111 classes.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 126, 126, 10)      280       
_________________________________________________________________
layer_normalization_4 (Layer (None, 126, 126, 10)      252       
_________________________________________________________________
conv2d_28 

Sep-28 - Mix up the training/validation data. Maybe using only noise data for train messes up non-noise validation.

Sep-30 - Changing data helped to get good validation accuracy results, but validation loss is stuck at ~.44, so maybe simplifing the model could help.

Oct-1 - Simplifing the model gave worst results, so maybe making it more complex should work? Using a VGG-16 based model (But with more complex structure) might be a good idea.

Oct-6 - VGG-16 model by itself gave a deadlock at .009 of val_acc, so probably the best approach is going back to the manually optimized model and keep tunning it.